In [ ]:
library(Seurat)
library(tidyverse)
library(cowplot)
library(patchwork)
library(ggplot2)
library(sWGCNA)
library(hdWGCNA)
library(dplyr)
library(doParallel)
library(edgeR)
library(ggpubr) 
library(lsr) 
library(viridis)
library(Hmisc)
library(pheatmap)
library(grid)
library(RColorBrewer)
library(gridExtra)
library(clusterProfiler)
library(GO.db)
library("org.Hs.eg.db")
library(ggrepel)
library(ggraph)
library(tidygraph)
library(stringr)

## Prepare hdWGCNA metacell object

In [ ]:
seurat_obj = readRDS('./scrna_lecMicro_cleaned_integrated_sctransform.RDS')




In [ ]:
DefaultAssay(seurat_obj) = 'RNA'

seurat_obj <- SetupForWGCNA(
  seurat_obj,
  gene_select = "fraction", 
  fraction = 0.05, # fraction of cells that a gene needs to be expressed
  wgcna_name = "lecMicro" 
)

In [ ]:
# construct metacells  in each group
seurat_obj <- MetacellsByGroups(target_metacells = 200,
  seurat_obj = seurat_obj,
  group.by = c("hash.ID"), # to group by
  reduction = 'pca', # dimensionality reduction to perform KNN on
  k = 25,
  max_shared = 10, 
  ident.group = 'hash.ID' #  Idents of the metacell seurat object
)



In [ ]:
seurat_obj <- NormalizeMetacells(seurat_obj)


In [ ]:
seurat_obj <- SetDatExpr(
  seurat_obj,
  group_name = unique(seurat_obj@meta.data$hash.ID), 
  group.by='hash.ID',
  assay = 'RNA', # using RNA assay
  slot = 'data' 
)

## Perform WGCNA

In [ ]:
# Test different soft powers:
seurat_obj <- TestSoftPowers(
  seurat_obj,
  networkType = 'signed hybrid' 
)

# plot the results:
plot_list <- PlotSoftPowers(seurat_obj)

# assemble with patchwork
wrap_plots(plot_list, ncol=2)

In [ ]:
power_table <- GetPowerTable(seurat_obj)
head(power_table)

In [ ]:
seurat_obj <- ConstructNetwork(
  seurat_obj,
  soft_power = 3,
  consensus = FALSE,
  overwrite_tom = TRUE,
  wgcna_name = NULL,
  blocks = NULL,
  maxBlockSize = 30000 ,
  randomSeed = 12345,
  corType = "pearson",
  consensusQuantile = 0.25,
  networkType = "signed hybrid",
  reassignThreshold = 0,
  TOMType = "signed",
  TOMDenom = "min",
  scaleTOMs = TRUE,
  scaleQuantile = 0.9,
  sampleForScaling = TRUE,
  sampleForScalingFactor = 1000,
  useDiskCache = TRUE,
  chunkSize = NULL,
  deepSplit = 4,
  pamStage = FALSE,
  detectCutHeight = 0.995,
  minModuleSize = 10,
  mergeCutHeight = 0,
  saveConsensusTOMs = TRUE
)

In [ ]:
PlotDendrogram(seurat_obj, main='LecMicro_human hdWGCNA Dendrogram')
modules <- GetModules(seurat_obj) 
table((modules$module))

length(unique(modules$module))

In [ ]:
seurat_obj <- ModuleEigengenes(
 seurat_obj,
)

In [ ]:
MEs <- GetMEs(seurat_obj, harmonized=FALSE)

seurat_obj <- ModuleConnectivity(
  seurat_obj
)

In [ ]:
PlotKMEs <- function(
  seurat_obj,
  n_hubs=10,
  text_size=2,
  ncol = 5,
  plot_widths = c(3,2),
  wgcna_name = NULL
){

  if(is.null(wgcna_name)){wgcna_name <- seurat_obj@misc$active_wgcna}

  modules <- GetModules(seurat_obj, wgcna_name) %>% subset(module != 'grey')
  mods <- levels(modules$module); mods <- mods[mods != 'grey']
  mod_colors <- modules %>% subset(module %in% mods) %>%
    dplyr::select(c(module, color)) %>%
    distinct


  #get hub genes:
  hub_df <- do.call(rbind, lapply(mods, function(cur_mod){
    print(cur_mod)
    cur <- subset(modules, module == cur_mod)
    cur <- cur[,c('gene_name', 'module', paste0('kME_', cur_mod))]
    names(cur)[3] <- 'kME'
    cur <- dplyr::arrange(cur, kME)
    top_genes <- cur %>% dplyr::top_n(n_hubs, wt=kME) %>% .$gene_name
    cur$lab <- ifelse(cur$gene_name %in% top_genes, cur$gene_name, "")
    cur
  }))
  head(hub_df)

  plot_list <- lapply(mods, function(x){
    print(x)
    cur_color <- subset(mod_colors, module == x) %>% .$color
    cur_df <- subset(hub_df, module == x)
    top_genes <- cur_df %>% dplyr::top_n(n_hubs, wt=kME) %>% .$gene_name
    p <- cur_df %>% ggplot(aes(x = reorder(gene_name, kME), y = kME)) +
      geom_bar(stat='identity', width=1, color = cur_color, fill=cur_color) +
      ggtitle(x) +
      #xlab(paste0('kME_', x)) +
      theme(
        axis.ticks.x = element_blank(),
        axis.text.x = element_blank(),
        plot.title = element_text(hjust=0.5),
        axis.title.x = element_blank(),
        axis.line.x = element_blank(), 
          title = element_text(size = 11.5)
      )
    p_anno <- ggplot() + annotate(
      "label",
      x = 0,
      y = 0,
      label = paste0(top_genes, collapse="\n"),
      size=text_size,
      fontface = 'italic',
      label.size=0
    ) + theme_void()
    patch <- p + p_anno + plot_layout(widths=plot_widths)
    patch
  })

  wrap_plots(plot_list, ncol=ncol)

}

In [ ]:
p <- PlotKMEs(seurat_obj, ncol=4, text_size = 3,   n_hubs=7)


In [ ]:
modules <- GetModules(seurat_obj) 
table((modules$module))

In [ ]:
write.csv(modules, 'modules_hdWGCNA_lecMicro.csv')

In [ ]:
micro.combined4 <- readRDS('./scrna_lecMicro_cleaned_integrated_sctransform.RDS')

## Plot modules

In [ ]:
ModuleNetworkPlot(
  seurat_obj,
  outdir = 'ModuleNetworks',  n_inner = 8,
  n_outer = 12,
)

In [ ]:
gene_names = GetHubGenes(seurat_obj, n_hubs =10)$gene_name

In [ ]:
modules <- GetModules(seurat_obj) %>% 
  subset(module != 'grey') %>% 
  mutate(module = droplevels(module))

mods <- levels(modules$module)
TOM <- GetTOM(seurat_obj)

# get module colors for plotting 
mod_colors <- dplyr::select(modules, c(module, color)) %>% distinct()
mod_cp <- mod_colors$color; names(mod_cp) <- as.character(mod_colors$module)

# get genes in this pathway 
cur_genes <- gene_names

# subset the TOM 
cur_TOM <- TOM[cur_genes,cur_genes] 

In [ ]:
graph <- cur_TOM %>% 
  igraph::graph_from_adjacency_matrix(mode='undirected', weighted=TRUE) %>% 
  tidygraph::as_tbl_graph(directed=FALSE) %>% 
  tidygraph::activate(nodes) 

p <- ggraph(graph) + 
  geom_edge_link(color='grey', alpha=0.2) + 
  geom_node_point(color='black') +
  geom_node_label(aes(label=name), repel=TRUE, max.overlaps=Inf, fontface='italic') 



In [ ]:
# only keep the upper triangular part of the TOM:
cur_TOM[upper.tri(cur_TOM)] <- NA

# cast the network from wide to long format
cur_network <- cur_TOM %>% 
  reshape2::melt() %>% 
  dplyr::rename(gene1 = Var1, gene2 = Var2, weight=value) %>%
  subset(!is.na(weight))

# get the module & color info for gene1
temp1 <- dplyr::inner_join(
  cur_network,
  modules %>% 
    dplyr::select(c(gene_name, module, color)) %>% 
    dplyr::rename(gene1 = gene_name, module1=module, color1=color),
  by = 'gene1') %>% dplyr::select(c(module1, color1))

# get the module & color info for gene2
temp2 <- dplyr::inner_join(
  cur_network,
  modules %>% 
    dplyr::select(c(gene_name, module, color)) %>% 
    dplyr::rename(gene2 = gene_name, module2=module, color2=color),
  by = 'gene2') %>% dplyr::select(c(module2, color2))

# add the module & color info 
cur_network <- cbind(cur_network, temp1, temp2)

# set the edge color to the module's color if they are the two genes are in the same module 
cur_network$edge_color <- ifelse(
  cur_network$module1 == cur_network$module2, 
  as.character(cur_network$module1),
  'grey')

# keep this network before subsetting
cur_network_full <- cur_network 

In [ ]:
options(repr.plot.width=30, repr.plot.height=5)

# subset to only keep edges between genes in the same module
cur_network <- cur_network_full %>% 
  subset(module1 == module2)

# make the graph object with tidygraph
graph <- cur_network %>% 
  igraph::graph_from_data_frame() %>%
  tidygraph::as_tbl_graph(directed=FALSE) %>% 
  tidygraph::activate(nodes)

# add the module name to the graph:
V(graph)$module <- modules[V(graph)$name,'module']

# get the top 25 hub genes for each module
hub_genes <- GetHubGenes(seurat_obj, n_hubs=25) %>% .$gene_name
V(graph)$hub <- ifelse(V(graph)$name %in% hub_genes, V(graph)$name, "")

# make the plot with gggraph
p <- ggraph(graph) + 
  geom_edge_link(aes(alpha=weight, color=edge_color)) + 
  geom_node_point(aes(color=module)) +
  geom_node_label(aes(label=hub, color = module), repel=TRUE, max.overlaps=Inf, fontface='italic') +
  scale_colour_manual(values=mod_cp) +
  scale_edge_colour_manual(values=mod_cp)+ theme_nothing() +
  NoLegend() 




In [ ]:
options(repr.plot.width=4, repr.plot.height=39)


plots <- list()


interest = c("green", "magenta", "turquoise", "brown", "greenyellow", "blue", "pink", "black", "salmon", "red", "yellow")
# Replace this with your actual process of creating each graph
for (i in 1:length(interest)) {
  # Assuming `cur_network` is defined and updated for each graph creation
  graph <- subset(cur_network, module1 == interest[i]) %>% 
    igraph::graph_from_data_frame() %>%
    tidygraph::as_tbl_graph(directed = FALSE) %>% 
    tidygraph::activate(nodes)
  
  # Add the module name to the graph
  V(graph)$module <- modules[V(graph)$name, 'module']
  
  # Get the top 25 hub genes for each module
  hub_genes <- GetHubGenes(seurat_obj, n_hubs = 25) %>% .$gene_name
  V(graph)$hub <- ifelse(V(graph)$name %in% hub_genes, V(graph)$name, "")
  
  # Make the plot with ggraph
  p <- ggraph(graph) + 
    geom_edge_link(aes(alpha = weight, color = edge_color)) + 
    geom_node_point(aes(color = module, size = 3)) +
    geom_node_label(aes(label = hub), repel = TRUE, max.overlaps = Inf, fontface = 'italic', size = 6) +
    scale_colour_manual(values = mod_cp) +
    scale_edge_colour_manual(values = mod_cp) +
    theme_void() + # use theme_void() instead of theme_nothing() to avoid errors
    theme(legend.position = "none") # use theme(legend.position = "none") instead of NoLegend()
  
  # Store the plot in the list
  plots[[i]] <- p
}

plots <- lapply(plots, function(p) p + theme(plot.margin = margin(5, 5, 5, 5))) 
grid.arrange(grobs = plots, nrow = 11, ncol = 1)

## Perform enrichment analysis of modules


In [ ]:
# store modules as list 

# Initialize an empty list to store modules
module_list <- list()

# Loop through unique module names and extract genes associated with each module
for (module in (modules$module)) {
  module_genes <- modules$gene_name[modules$module == module]
  module_list[[module]] <- module_genes
}



In [ ]:
#GO BPs enrichment 

results_BP = list()

for (i in (unique(res$module))) {

    gene_list <- (subset(modules, module == i))$gene_name
    
    go_enrich <- enrichGO(gene          = gene_list,
                          universe     = row.names(micro.combined4),
                          OrgDb        = org.Hs.eg.db,
                          keyType      = "SYMBOL",
                          ont          = "BP",   # Biological Process
                          minGSSize = 10,
                          pAdjustMethod = "BH",
                          qvalueCutoff = 0.2)
    
    # Store the results in a list
    results_BP[[i]] <- go_enrich
    
}

In [ ]:
#GO MFs enrichment 

results_MF = list()

for (i in (unique(res$module))) {
    

    gene_list <- (subset(res, module == i))$gene_name
    
    # Perform GO enrichment analysis
    go_enrich <- enrichGO(gene          = gene_list,
                          universe     = row.names(micro.combined4),
                          OrgDb        = org.Hs.eg.db,
                          keyType      = "SYMBOL",
                          ont          = "MF",   # Biological Process
                        minGSSize = 10,
                          pAdjustMethod = "BH",
                          qvalueCutoff = 0.2)
    
    # Store the results in a list
    results_MF[[i]] <- go_enrich
    
}

In [ ]:
df_full <- data.frame()

for (i in 1:length(results)){
    df1 = results_BP[[i]]@result
    df2=results_MF[[i]]@result
    
    df1$module = names(results)[[i]]
    df2$module = names(results)[[i]]

    df1$Ontology = ("BP")
    df2$Ontology = ("MF")

    df1$Description2 = paste(df1$Description, "(BP)")
    df2$Description2 = paste(df2$Description, "(MF)")

    df = rbind(df1, df2)
    df <- df[order(-df$p.adjust, decreasing = TRUE), ]

    df = head(df, 20)
    GO_full = rbind(df_full, df)
}


In [ ]:

df_GO_save =GO_full[c('module','Ontology','ID','Description','GeneRatio','BgRatio',
             'pvalue','p.adjust','qvalue','geneID','Count')]



In [ ]:
for (i in 1:length(module_list)){
    name = (names(module_list[i]))
    name = paste(name,".tab", sep = "") 
    print(name)
    assign(name, data.frame(geneID = ((module_list[i]))[[1]], set = rep(names(((module_list[i]))))))
    
}

# ATM enrichment

In [ ]:
law = (read.csv('../../ATM_genes/Lawrence2024_hs_syngo', header = 0))$V1
ham = (read.csv('../../ATM_genes/Hammond2019_hs_syngo', header = 0))$V1
li = (read.csv('../../ATM_genes/Li2019_hs_syngo', header = 0))$V1


In [ ]:
# Function to calculate overlaps and p-values using hypergeometric test
calculate_overlap <- function(gene_list1, gene_list2, total_genes) {
  overlap_genes <- length(intersect(gene_list1, gene_list2))
  num_genes_list1 <- length(gene_list1)
  num_genes_list2 <- length(gene_list2)

  # Perform the hypergeometric test
  p_value <- phyper(q = overlap_genes - 1, 
                    m = num_genes_list1, 
                    n = total_genes - num_genes_list1, 
                    k = num_genes_list2, 
                    lower.tail = FALSE)
  
  return(list(overlap = overlap_genes, p_val = p_value))
}

# Set the universe of genes (total genes in the dataset)
total_genes <- length(row.names(micro.combined4))

# Function to process each gene set and calculate overlaps for multiple modules
process_gene_sets <- function(gene_list, module_list, set_name) {
  overlap_list <- list()
  p_val_list <- list()

  for (i in 1:length(module_list)) {
    gene_list2 <- module_list[[i]]
    
    # Calculate overlap and p-value
    result <- calculate_overlap(gene_list, gene_list2, total_genes)
    overlap_list[[names(module_list)[i]]] <- result$overlap
    p_val_list[[names(module_list)[i]]] <- result$p_val
  }

  # Combine results into a data frame
  overlap_df <- as.data.frame(t(as.data.frame(overlap_list)))
  p_val_df <- as.data.frame(t(as.data.frame(p_val_list)))
  overlap_df$p_val <- p_val_df$V1
  overlap_df$set <- set_name
  
  # Set proper column names
  colnames(overlap_df) <- c("overlap", "p.val", "set")
  return(overlap_df)
}

# Calculate overlaps for Li et al. and Lawrence et al.
li_df <- process_gene_sets(li, module_list, "li")
law_df <- process_gene_sets(law, module_list, "law")

# Combine the two data frames
df <- rbind(li_df, law_df)

# Adjust p-values using BH correction
df$p_adj <- p.adjust(df$p.val, method = "BH")

# Restructure the data frame for final output
df_li <- subset(df, set == "li")[c('overlap', 'p_adj')]
colnames(df_li) <- c("Li_overlap", "Li_p.adj")

df_li$Lawrence_overlap <- subset(df, set == "law")$overlap
df_li$Lawrence_p.adj <- subset(df, set == "law")$p_adj

# Add the module names and reorder columns
df_li$module <- row.names(df_li)
df_li <- df_li[c("module", "Li_overlap", "Li_p.adj", "Lawrence_overlap", "Lawrence_p.adj")]

# Write the results to a file
write.table(df_li, "Lec_hdWGCNA_Modules_ATM_Overlap.tsv", sep = '\t', row.names = FALSE)


In [ ]:

# Example sets
set1 <- law
set2 <- li
set3 <- module_list[["pink"]]


# Calculate the areas for the 3-way Venn diagram
venn.plot <- draw.triple.venn(
  area1 = length(set1),
  area2 = length(set2),
  area3 = length(set3),
  n12 = length(intersect(set1, set2)),
  n13 = length(intersect(set1, set3)),
  n23 = length(intersect(set2, set3)),
  n123 = length(Reduce(intersect, list(set1, set2, set3))),
  category = c("Lawrence", "Li", "Pink Module"),
  fill = c("red", "blue", "pink"),
  lty = "dashed",
  cex = 2,
  cat.cex = 2,
  cat.col = c("red", "blue", "pink")
)

# Display the Venn diagram
grid.draw(venn.plot)

## GSEA of modules in scRNA-seq (IgG1 vs. Lec)

In [ ]:
Idents(micro.combined4) <- micro.combined4@meta.data$Treatment
DefaultAssay(micro.combined4) <- 'SCT'

Lec_iGg1 = read.csv( '../110.ProcessFiles_B1B2/Lec_iGg1_DE_onlyFIRE_SCT_30APR2024.csv')
row.names(Lec_iGg1) = Lec_iGg1$X

In [ ]:
# Set plot dimensions
options(repr.plot.width = 18, repr.plot.height = 10)

# Function to create volcano plot
volPlot <- function(markers, colors) {
  
  markers$X = rownames(markers)

  ggplot(markers, aes(x = avg_log2FC, y = -log10(p_val_adj),  
                      label = ifelse(p_val_adj < 0.05 & abs(avg_log2FC) > 0.05, X, ""),
                      fill = factor(ifelse(
                        avg_log2FC < 0 & p_val_adj < 0.05, "Down-regulated",
                        ifelse(avg_log2FC > 0 & p_val_adj < 0.05, "Up-regulated", "NS")
                      )))) +  

    scale_fill_manual(name = "Direction (padj < 0.05)",
                      values = colors) + 
    
    geom_point(size = 5, shape = 21, color = "grey2", stroke = 0.2) +
    
    geom_hline(yintercept = 1.2, color = "red", linetype = "dashed") +
    geom_vline(xintercept = 0, color = "black", linetype = "dashed") +
    
    # Styling for theme
    theme_bw(base_size = 22) + 
    theme(
      plot.title = element_text(hjust = 0.5, size = 24, face = "bold"),
      axis.title = element_text(size = 18),
      axis.text = element_text(size = 16),
      legend.position = "right",
      legend.title = element_text(size = 16),
      legend.text = element_text(size = 14),
      panel.grid = element_blank(),
      panel.border = element_blank(),
      strip.text = element_text(size = 16)
    ) +
    
    # Add labels to significant genes
    geom_label_repel(
      size = 6, 
      max.overlaps = 30, 
      fill = NA, 
      force = 0.4, 
      nudge_y = -0.1, 
      box.padding = 0.5,
      label.size = 3,
      label.padding = 0.3
    ) +
    
    # Set labels, title, legend
    xlab("\nAverage log2 Fold Change") + 
    ylab("-log10(padj) \n") +
    guides(
      fill = guide_legend(order = 1),
      col = guide_legend(order = 2)
    )
}



In [ ]:
options(repr.plot.width=18, repr.plot.height=10)

volPlot(Lec_iGg1, c("black", "grey","red"))  + ggtitle("Lec vs. iGg1 (magenta)")

In [ ]:
# Prepare gene set data
els <- paste(names(module_list), ".tab", sep = "")
sets <- do.call(rbind, mget(els))
colnames(sets) <- c("geneID", "set")

# Reorganize the sets data frame
sets2 <- data.frame(set = sets$set, geneID = sets$geneID)

# Prepare the gene list from your differential expression data
df <- Lec_iGg1
df$X <- rownames(df)
original_gene_list <- df$avg_log2FC
names(original_gene_list) <- df$X

# Remove NA values and sort the gene list in decreasing order
gene_list <- na.omit(original_gene_list)
gene_list <- sort(gene_list, decreasing = TRUE)

# Perform GSEA
gsea_modules_sc <- GSEA(
  gene_list, 
  TERM2GENE = sets2, 
  eps = 0, 
  minGSSize = 3, 
  maxGSSize = 3000,
  pAdjustMethod = "BH",  
  pvalueCutoff = 0.05
)

# Sort GSEA results by normalized enrichment score (NES)
gsea_modules_sc@result <- gsea_modules_sc@result[order(gsea_modules_sc@result$NES), ]


In [ ]:

options(repr.plot.width=11, repr.plot.height=10)

# Assuming res_gsea$Description contains color names or hex codes
res_gsea = gsea_modules_sc@result

# Create the column reflecting NES /sig values
res_gsea$Treatment = ifelse(res_gsea$NES > 0 & res_gsea$p.adjust < 0.05, "Lec", 
                   ifelse(res_gsea$NES < 0 & res_gsea$p.adjust < 0.05, "iGg1", "C"))


res_gsea$OutlineColor <- ifelse(res_gsea$NES > 0, "Lec", "IgG1")
res_gsea$OutlineColor <- factor(res_gsea$OutlineColor, levels = c("Lec", "IgG1"))

res_gsea$Significance <- ifelse(res_gsea$qvalue < 0.001, "***",
                     ifelse(res_gsea$qvalue < 0.01, "**",
                     ifelse(res_gsea$qvalue < 0.05, "*", "")))


ggplot(res_gsea, aes(x = reorder(ID, -NES), y = NES, fill = Description, color = OutlineColor)) +
      geom_bar(stat = "identity", width = 0.8, size = 0.8, alpha = 0.8) + 
      geom_text(aes(label = Significance, y = NES - 0.3 * sign(NES)), 
                size = 6, fontface = "bold") +
    geom_hline(yintercept = 0,size = 1)+
      scale_fill_identity() +  # Use colors from Description column
      scale_color_manual(values = c("Lec" = "red2", "IgG1" = "black")) +  
      
      scale_x_discrete(expand = expansion(add = .5)) +
      scale_y_continuous(
        breaks = seq(
          floor(min(res_gsea$NES)), ceiling(max(res_gsea$NES)),
          ceiling((ceiling(max(res_gsea$NES)) - floor(min(res_gsea$NES))) / 6)
        )       ) +
      coord_flip() + 
      theme_bw(base_size = 22) +   
        theme(
        legend.position = "right",
        legend.key.size = unit(1.5, "lines"),
        legend.text = element_text(size = 22),
        axis.text = element_text(size = 22),
        axis.title = element_text(size = 24), 
        plot.title = element_text(size = 24, face = 'bold', hjust = 0.6, 
                                 margin = margin(b = 20)), 
        plot.margin = margin(10,50,10, 50), 
        legend.title = element_text(size = 24)

      ) +  
      xlab("")+
    labs(title = "GSEA Lec vs. IgG1: WGCNA modules",  y = "NES", color = "Direction") + 
      labs(color = NULL, fill = NULL) +  
    labs(fill = NULL) +  
      guides(fill = "none", color = 'none')  
